In [1]:
import numpy as np
import re
import string
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer
%matplotlib inline

In [2]:
df = pd.read_excel('reviews.xlsx')
df.head()

,target,movie_link,author_id,author_name,review_subtitle,review_text
0,good,/film/178563/,/user/311869/,sochi2014,NaN,Настоящее кино. Наше кино. Прекрасная работа с...
1,good,/film/178563/,/user/1591566/,Альберт Попов,Предтеча многого,"Мало того, что сериал «Империя под ударом» (12..."
2,good,/film/178563/,/user/331395/,LennoxL,Срез жизни общества начала 20 го века,Период от реформ Александра Освободителя до пе...
3,good,/film/178563/,/user/4379604/,Ana_K,"Пусть мир погибнет, но восторжествует правосудие",Атмосферный сериал о Российской Империи начала...
4,neutral,/film/178563/,/user/178788/,Skept,Террор  Зло с непредсказуемыми последствиями,Самая большая беда современного общества это т...


In [3]:
df.shape

(80284, 6)

In [4]:
df.groupby(['target']).count()

,movie_link,author_id,author_name,review_subtitle,review_text
target,,,,,
bad,7281,7281,7281,5475,7281
good,60189,60189,60187,46500,60187
neutral,12814,12814,12814,8225,12814


In [13]:
print(negative_reviews[1])

Сразу хотелось бы честно признаться: не досмотрел до конца даже первую серию. Простите, не смог. Можно сколько угодно рассуждать о ценности данного фильма, об актерских работах и подборе звезд, но не заметить факта, что отсебятина хлещет через край, невозможно.

Если есть люди, прочитавшие книгу, они поймут моё отношение к данной экранизации. С самого начала идут какие-то придуманные диалоги и реплики, которых не должно быть по сути. Задача сценаристов была сложная, но её облегчили донельзя. Ни одна сцена не избежала изменений. Прежде чем снимать, нужно было, во-первых, несколько раз прочитать роман, во-вторых, глубоко понять его психологию. О чем можно говорить, если у Фантины одна подруга, она знает о беременности и вовсеуслышание это говорит? Господа, у Гюго и близко такого не было! Минус 5 из плюс 10 возможных.

Обратимся к образу епископа. Он в книге никогда не поручал другому то, что мог сделать сам, он описан совершенным ангелом во плоти (со своими маленькими мсинусами, естестве

In [6]:
df_80 = (df.shape[0] * 80) // 100

positive_reviews = df[df['target'] == 'good']['review_text'].astype('str').to_numpy()
negative_reviews = df[df['target'] == 'bad']['review_text'].astype('str').to_numpy()
neutral_reviews = df[df['target'] == 'neutral']['review_text'].astype('str').to_numpy()

example_positive_review = positive_reviews[0]
example_negative_review = negative_reviews[0]
example_neutral_review = neutral_reviews[0]

test_pos = positive_reviews[df_80:]
train_pos = positive_reviews[:df_80]

test_neg = negative_reviews[df_80:]
train_neg = negative_reviews[:df_80]

test_neu = neutral_reviews[df_80:]
train_neu = neutral_reviews[:df_80]

train_x = np.concatenate((train_pos, train_neg, train_neu))
test_x = np.concatenate((test_pos, test_neg, test_neu))

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)),  axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [26]:
# def review_process(review):
#     review = re.sub(r'[^0-9а-яА-ЯёЁ]+', ' ', review)
#     tokenizer = TweetTokenizer()
#     review_tokenized = tokenizer.tokenize(review)
#     stopwords = nltk.corpus.stopwords.words("russian")
#     review_processsed = [word for word in review_tokenized if word not  in stopwords and word not in string.punctuation]
#     stemmer = PorterStemmer() 
#     review_after_stem = []
#     for word in review_processsed:
#         word = stemmer.stem(word)
#         review_after_stem.append(word)
#     return review_after_stem

def clean_text(text):
    text = re.sub(r'[^0-9а-яА-ЯёЁ]+', ' ', text)
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = nltk.corpus.stopwords.words("russian")
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [ ]:
# clean text data
reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))

In [22]:
pos_words, neg_words, freq_pos, freq_neg = np.array([]), np.array([]), {}, {}

for review in positive_reviews[:1000]:
    review = review_process(review)
    pos_words = np.append(pos_words, review)

for word in pos_words:
    if (word, 1) not in freq_pos:
        freq_pos[(word, 1)] = 1
    else:
        freq_pos[(word, 1)] = freq_pos[(word, 1)] + 1

for review in negative_reviews[:1000]:
    review = review_process(review)
    neg_words = np.append(neg_words, review)
        
for word in neg_words:
    if (word,0) not in freq_neg:
        freq_neg[(word,0)] = 1
    else:
        freq_neg[(word,0)] = freq_neg[(word,0)] + 1
        
freqs_dict = dict(freq_pos)
freqs_dict.update(freq_neg)

In [23]:
def features_extraction(review, freqs_dict):
    word_l = review_process(review)
    x = np.zeros((1, 3))
    x[0,0] = 1 
    for word in word_l:
        try:
            x[0,1] += freqs_dict[(word, 1)]
        except:
            x[0,1] += 0
        try: 
            x[0,2] += freqs_dict[(word, 0.0)]
        except:
            x[0,2] += 0
    assert(x.shape == (1, 3))
    return x

X = np.zeros((len(train_x[:1000]), 3))
    
for i in range(len(train_x[:1000])):
    X[i, :]= features_extraction(train_x[i], freqs_dict)

In [24]:
def sigmoid(x): 
    h = 1/(1 + np.exp(-x))
    return h

def gradientDescent_algo(x, y, theta, alpha, num_iters):
    m = x.shape[0]
    for i in range(0, num_iters):
        z = np.dot(x,theta)
        h = sigmoid(z)
        J = -1/m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))
        theta = theta - (alpha/m) * np.dot(x.T,h-y)
    J = float(J)
    return J, theta

In [25]:
X = np.zeros((len(train_x[:1000]), 3))
for i in range(len(train_x[:1000])):
    X[i, :] = features_extraction(train_x[i], freqs_dict)
Y = train_y
J, theta = gradientDescent_algo(X, Y, np.zeros((3, 1)), 1e-9, 1500)

ValueError: shapes (1,67470) and (1000,1) not aligned: 67470 (dim 1) != 1000 (dim 0)

In [ ]:
def predict(tweet, freqs_dict, theta):
    x = features_extraction(tweet,freqs_dict)
    y_pred = sigmoid(np.dot(x,theta))
    return y_pred
def test_accuracy(test_x, test_y, freqs_dict, theta):
    y_hat = []
    for tweet in test_x:
        
        y_pred = predict(tweet, freqs_dict, theta)
        
        if y_pred > 0.5:
           
            y_hat.append(1)
        else:
            
            y_hat.append(0)
    m=len(y_hat)
    y_hat=np.array(y_hat)
    y_hat=y_hat.reshape(m)
    test_y=test_y.reshape(m)
    
    c=y_hat==test_y
    j=0
    for i in c:
        if i==True:
            j=j+1
    accuracy = j/m
    return accuracy
accuracy = test_accuracy(test_x, test_y, freqs_dict, theta)
print(accuracy)